# Building an Agent around a Query Pipeline

In this cookbook we show you how to build an agent around a query pipeline.

Agents offer the ability to do complex, sequential reasoning on top of any query DAG that you have setup. Conceptually this is also one of the ways you can add a "loop" to the graph.

Here we show you how to add an agent on top of a query pipeline.

## Setup Data

We use the chinook database as sample data. [Source](https://www.sqlitetutorial.net/sqlite-sample-database/).

In [17]:
!curl "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O ./chinook.zip
!unzip ./chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  3756k      0 --:--:-- --:--:-- --:--:-- 3926k
curl: (6) Could not resolve host: .
Archive:  ./chinook.zip
  inflating: chinook.db              


In [1]:
from llama_index import SQLDatabase
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:///chinook.db")
sql_database = SQLDatabase(engine)

In [2]:
from llama_index.query_pipeline import QueryPipeline

## Setup Text-to-SQL Query Engine / Tool

Now we setup a simple text-to-SQL tool: given a query, translate text to SQL, execute against database, and get back a result. 

In [35]:
from llama_index.query_engine import NLSQLTableQueryEngine
from llama_index.tools.query_engine import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, 
    tables=["albums", "tracks", "artists"], 
    verbose=True
)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)

### Define I/O Agent Components

To setup a query pipeline in an agent setting, we define two components:
- An `AgentInputComponent` that allows you to convert the agent inputs (Task, state dictionary) into a set of inputs for the query pipeline.
- An `AgentFnComponent` as an output processor: a general processor that allows you to take in the current Task, state, as well as any arbitrary inputs, and returns an output. In this cookbook we put this at the end to process the output and return a response. However, you can put this anywhere, wherever you need access to the Task and state.

Note that any function passed into `AgentFnComponent` and `AgentInputComponent` MUST include `task` and `state` as input variables, as these are inputs passed from the agent. 
The `AgentFnComponent` can also contain an arbitrary number of extra arguments.

In [52]:
from llama_index.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep,
)
from llama_index.agent import Task, AgentChatResponse
from llama_index.query_pipeline import (
    AgentInputComponent,
    AgentFnComponent
)
from llama_index.llms import MessageRole
from typing import Dict, Any, Optional, Tuple, List, cast

## Agent Input Component
## This is the component that produces agent inputs to the rest of the components
## Can also put initialization logic here.
def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict[str, Any]:
    """Agent input function.

    Returns:
        A Dictionary of output keys and values. If you are specifying
        src_key when defining links between this component and other
        components, make sure the src_key matches the specified output_key.
    
    """
    # initialize current_reasoning
    if "current_reasoning" not in state:
        state["current_reasoning"] = []
    reasoning_step = ObservationReasoningStep(observation=task.input)
    state["current_reasoning"].append(reasoning_step)
    return {"input": task.input}
    
agent_input_component = AgentInputComponent(fn=agent_input_fn)


## Agent Output Component
## Process reasoning step/tool outputs, and return agent response
def finalize_fn(
    task: Task, 
    state: Dict[str, Any], 
    reasoning_step: Any, 
    is_done: bool = False,
    tool_output: Optional[Any] = None
) -> Tuple[AgentChatResponse, bool]:
    """Finalize function.

    Here we take the latest reasoning step, and a tool output (if provided),
    and return the agent output (and decide if agent is done).
    
    """
    current_reasoning = state["current_reasoning"]
    current_reasoning.append(reasoning_step)
    # if tool_output is not None, add to current reasoning
    if tool_output is not None:
        observation_step = ObservationReasoningStep(observation=str(tool_output))
        current_reasoning.append(observation_step)
    if isinstance(current_reasoning[-1], ResponseReasoningStep):
        response_step = cast(ResponseReasoningStep, current_reasoning[-1])
        response_str = response_step.response
    else:
        response_str = current_reasoning[-1].get_content()

    # if is_done, add to memory
    # NOTE: memory is a reserved keyword in `state`, but you can add your own too
    if is_done:
        memory = state["memory"]
        memory.put(ChatMessage(content=task.input, role=MessageRole.USER))
        memory.put(ChatMessage(content=response_str, role=MessageRole.ASSISTANT))

    return AgentChatResponse(response=response_str), is_done


agent_output_component = AgentFnComponent(fn=finalize_fn)

### Define Agent Prompt

Here we define the agent component that generates a ReAct prompt, and after the output is generated from the LLM, parses into a structured object.

In [37]:
from llama_index.agent.react.formatter import ReActChatFormatter
from llama_index.query_pipeline import InputComponent, Link
from llama_index.llms import ChatMessage
from llama_index.tools import BaseTool

## define prompt function
def react_prompt_fn(
    task: Task,
    state: Dict[str, Any],
    input: str,
    tools: List[BaseTool]
) -> List[ChatMessage]:
    # Add input to reasoning
    chat_formatter = ReActChatFormatter()
    input_chat = chat_formatter.format(
        tools,
        chat_history=task.memory.get() + state["memory"].get_all(),
        current_reasoning=state["current_reasoning"]
    )
    print(f"CURRENT CHAT: {state['current_reasoning']}")
    return input_chat

react_prompt_component = AgentFnComponent(fn=react_prompt_fn, partial_dict={"tools": [sql_tool]})

### Define Agent Output Parser + Tool Pipeline

Once the LLM gives an output, we have a decision tree:
1. If an answer is given, then we're done. Process the output
2. If an action is given, we need to execute the specified tool with the specified args, and then process the output.

Tool calling can be done via the `ToolRunnerComponent` module. This is a standalone module that takes in a list of tools, and can be "executed" with the specified tool name (every tool has a name) and tool action.

In [53]:
from llama_index.query_pipeline import ToolRunnerComponent, IfElseComponent
from llama_index.llms import ChatMessage, ChatResponse
from llama_index.agent.react.output_parser import ReActOutputParser

## define tool pipeline
## Tool Runner Component
tool_runner_component = ToolRunnerComponent([sql_tool])
tool_qp = QueryPipeline(modules={
    "input": InputComponent(),
    "tool_runner": tool_runner_component,
    "agent_output": agent_output_component
}, verbose=True)
tool_qp.add_links([
    Link("input", "tool_runner", src_key="tool_name", dest_key="tool_name"),
    Link("input", "tool_runner", src_key="tool_input", dest_key="tool_input"),
    Link("input", "agent_output", src_key="reasoning_step", dest_key="reasoning_step"),
    Link("input", "agent_output", src_key="is_done", dest_key="is_done"),
    Link("tool_runner", "agent_output", dest_key="tool_output")
])



## define simple if-else decision module to only call tool if the agent isn't done
## otherwise skip and directly process output 
def react_output_fn(
    chat_response: ChatResponse,
) -> Tuple[bool, Dict[str, Any]]:
    """ReAct output function."""
    output_parser = ReActOutputParser()
    reasoning_step = output_parser.parse(chat_response.message.content)
    if reasoning_step.is_done:
        return True, {"reasoning_step": reasoning_step, "is_done": True}
    else:
        return False, {
            "tool_name": reasoning_step.action,
            "tool_input": reasoning_step.action_input,
            "reasoning_step": reasoning_step,
            "is_done": False
        }

react_output_component = IfElseComponent(
    fn=react_output_fn,
    choice1=agent_output_component,
    choice2=tool_qp
)

In [39]:
from llama_index.query_pipeline import QueryPipeline as QP
from llama_index.llms import OpenAI

qp = QP(
    modules={
        "agent_input": agent_input_component,
        "react_prompt": react_prompt_component,
        "llm": OpenAI(model="gpt-4-1106-preview"),
        "react_output": react_output_component,
    },
    verbose=True,
)
qp.add_link("agent_input", "react_prompt", dest_key="input")
qp.add_chain(["react_prompt", "llm", "react_output"])

### Visualize Query Pipeline

In [40]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)
net.show("agent_dag.html")

agent_dag.html


In [41]:
# TOOL Pipeline
# net = Network(notebook=True, cdn_resources="in_line", directed=True)
# net.from_nx(tool_qp.dag)
# net.show("agent_dag.html")

## Setup Agent around Text-to-SQL Query Pipeline

This is our way to setup an agent around a text-to-SQL Query Pipeline

In [42]:
from llama_index.agent import QueryPipelineAgentWorker, AgentRunner
from llama_index.callbacks import CallbackManager

agent_worker = QueryPipelineAgentWorker(qp)
agent = AgentRunner(agent_worker, callback_manager=CallbackManager([]))

In [43]:
agent_worker.agent_fn_components

[AgentFnComponent(partial_dict={'tools': [<llama_index.tools.query_engine.QueryEngineTool object at 0x157cef070>]}, fn=<function react_prompt_fn at 0x157cc29e0>, async_fn=None),
 AgentFnComponent(partial_dict={}, fn=<function finalize_fn at 0x1680031c0>, async_fn=None),
 AgentFnComponent(partial_dict={}, fn=<function finalize_fn at 0x1680031c0>, async_fn=None)]

## Run the Agent

Let's try the agent on some sample queries.

In [44]:
# response = agent.chat(
#     "What are the top modes of transporation fo the city with the higehest population?"
# )
# print(str(response))

In [45]:
# start task
task = agent.create_task("What are some tracks from the artist AC/DC? Limit it to 3")

In [46]:
step_output = agent.run_step(task.task_id)

> Running module agent_input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='e93cfd04-1dec-4121-9c32-c48285776de6' input='What are some tracks from the artist AC/DC? Limit it to 3' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method ...

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

CURRENT CHAT: [ObservationReasoningStep(observation='What are some tracks from the artist AC/DC? Limit it to 3')]
> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Too...

> Running module react_output with input: 
chat_response: assist

In [47]:
step_output = agent.run_step(task.task_id)

> Running module agent_input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='e93cfd04-1dec-4121-9c32-c48285776de6' input='What are some tracks from the artist AC/DC? Limit it to 3' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method ...

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

CURRENT CHAT: [ObservationReasoningStep(observation='What are some tracks from the artist AC/DC? Limit it to 3'), ActionReasoningStep(thought='I need to use a tool to help me answer the question.', action='sql_tool', action_input={'input': "Select track_name from music_database where artist_name = 'AC/DC' limit 3"}), ObservationReasoningStep(observation='The top 3 tracks by AC/DC are "For Those About To Rock (We Salute You)

In [48]:
step_output.is_last

True

In [49]:
response = agent.finalize_response(task.task_id)

In [50]:
print(str(response))

The top 3 tracks by AC/DC are "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up".


## Setup Agent with Tool Use

User query -> reasoning -> output parsing + tool use -> add to agent state